# P5: Identify Fraud from Enron Email
##### Udacity Data Analyst Nanodegree
##### Kellen Proctor
##### 29 June 2017

# Initial Exploration

I began by exploring the features of data_dict, to get a full understanding of what I am dealing with.

## 1. Project Summary

The goal is simply to identify persons of interest ("POI") at Enron, ie those that were likely to have organized, perpetrated, and profited from the ongoing fraudulent activities at the firm. By the way, I watched the documentary, and being from California, what they did was just unconscionable. I have no idea how they were able to get access to the Power Grid, being able to shut off power plants and all, but it's kinda messed up that more people didn't get indicted. Regardless... The dataset has a wealth of financial and email related features. It would be difficult to identify POIs by hand. Instead, we can use machine learning algorithms to try to come up with a way to determine POIs programmatically.

### Data Exploration
There are 146 datapoints and 18 of those are POIs. The dataset contains 20 different features, split bewteen 14 financial and 6 email related features, for the people we are investigating. There's also a label - "poi" - that we can use for identifying persons of interest. Interestingly enough, only the features 'expenses', 'other', 'total_payments', 'total_stock_value', (and with 1 exception) 'restricted_stock', and 'salary' had valid data for all POIs. I may choose to use just these features in feature selection, but I'd like to keep all the features for now, and see what the results of using feature selection algorithms show.

### Outliers
I simply printed out a list of the keys of data_dict. Based on the outliers mini-project, and a couple forum entries, I found "TOTAL" and "THE TRAVEL AGENCY IN THE PARK." Both were not POIs, from looking at their data, they either skewed the total data, or had none. So I removed both of these from the dataset. I chose to leave the remaining entries in data_dict, as they represent actual people. Further, the dataset is already very tiny, so I don't want to lose more information if I don't have to.

After running a few scripts to analyze the remaining persons and features, I chose to also remove 'LOCKHARD EUGENE E', who basically had no data.


## 2. Feature Selection
To start, I added all the features available, with the exception of emails.

I created fractional features for payments, stock value, and email related features. This was based off of a forum post I read, but I figured it also may make certain features easier to compare between individuals, since some features like salary and bonus won't be so skewed by outlying individuals.

I then removed 'loan_advances', 'director_fees', and 'restricted_stock_deferred', as well as their fractional equivalents, from the features list. These features had so little data overall, I figured they would contribute very little to the overall model.

After experimenting with various values of k for SelectKBest with a naive bayes classifier, I ended up using the following list of features ('salary', 'bonus_fraction', 'long_term_incentive', 'deferred_income', 'deferral_payments', 'other', 'expenses', 'total_payments', 'exercised_stock_options', 'restricted_stock', 'total_stock_value', 'from_this_person_to_poi', 'to_messages', 'from_poi_to_this_person', 'from_messages', 'shared_receipt_with_poi'), which would be trimmed down in an optimized GridSearchCV pipeline, before being fed to the Naive Bayes classifier I ended up using. The top 5 features being used in the naive bayes classifier, and their respective scores, are here:

| Feature | Score | p-value|
|---|---|---|
|'total_payments'| 24.815079733218194| 1.8182048777865317e-06|
|'restricted_stock'| 24.182898678566879| 2.4043152760436886e-06|
|'salary'| 20.715596247559954| 1.1403848516905529e-05|
|'long_term_incentive'| 11.458476579280369| 0.00092203670846723062|
|'bonus_fraction'| 9.9221860131898225| 0.0019941812453537077|

## 3. Algorithm

I ended up using the Naive Bayes classifier (GaussianNB).

I originally set up a script to run a classification report for Gaussian Naive Bayes, Decision Trees, Random Forest, and AdaBoost classifiers on the data split using train_test_split with a test size of 20%. The results of this are here:

| Algorithm    | Precision | Recall | F1 Score | Support |
|--------------|-----------|--------|----------|---------|
|    GaussNB   |    0.92   |  0.90  |   0.91   |    29   |
| DecisionTree |    0.86   |  0.79  |   0.82   |    29   |
| RandomForest |    0.86   |  0.90  |   0.88   |    29   |
|   Adaboost   |    0.86   |  0.83  |   0.84   |    29   |

Based on these results, I initially decided to use the Random Forest algorithm, and proceeded to tune the parameters of that, which I will explain below. However, I was unable to achieve 0.3 recall on the tester.py script with any configuration. On a whim, I ran Naive Bayes, and lowe and behold, it worked just fine. So my final submission uses Naive Bayes.

## 4. Algorithm Tuning

Tuning an algorithm is the process of adjusting the parameters so as to optimize your algorithm to the dataset it is being run on. We are trying to get the algorithm to the sweetspot between high bias and high variance. If our algorithm is too biased to the dataset, then it isn't training well, which results in poor performance. If our algorithm has too much variance, then it we may have overfit the algorithm to the training data. This means it won't generalize well, making it a poor predictor moving forward. Either way, we end up with an algorithm that reacts poorly with real world data outside of our training set. We are trying to get it to a point where the algorithm generalizes well to new data, but isn't too simple. That is the goal of tuning.

I tuned the RandomForest algorithm by running it through a GridSearchCV pipeline, and adjusting the parameters for n_estimators, max_features, and criterion, as well as the SearchKBest k-values. I also implemented three different scoring functions for GridSearchCV (accuracy_score, precision_score, and recall_score). I took the 3 best parameter settings for each scoring function, and proceeded with validation and evaluation as below.

## 5. Validation

Validation is the process of running your trained algorithm on test dataset that is different from the data that you used to train the algorithm in the first place. Our goal is to give an estimate of the performance of the algorithm on an independent dataset and to check that we're not overfitting our algorithm, which would result in a poorly functioning algorithm. I validated my data by splitting the data set using train_test_split with a test size of 20%. I did this for both the RandomForest and Naive Bayes algorithms

## 6. Usage of Evaluation Metrics

The three evaluation metrics I primarily used are accuracy, precision, and recall. My average performace for those metrics with the RandomForest and Naive Bayes algorithms are the following:

| Algorithm    | Accuracy | Precision | Recall |
|--------------|----------|-----------|--------|
| RandomForest | 0.85     | 0.35      | 0.15   |
| NaiveBayes   | 0.84     | 0.40      | 0.31   |

With regards to accuracy, on the most basic level, both algorithms were able to predict correctly, roughly 85% of the time, whether or not a person in the dataset was a Person of Interest (POI). However, accuracy just tells us if the prediction matches the actual label in the test dataset. We can further break that number down, and determine how many times we correctly predicted positive and negative values, and how many times our predictions were wrong positively or negatively. Precision tells us then how many times we were correct in predicting someone to be a POI, out of all positive predictions of people being POIs. With a precision rate of 0.40 for NaiveBayes and 0.35 for RandomForest, we were only good at positively identifying a POI at best 40% of the time.  As for Recall, we're trying to figure out how many times we predicted someone being a POI, and they actually were a POI. For recall, our best was .31 with the naive bayes algorithm. These results aren't actually that great. Overall, we're decent at predicting the outcome, but we're not that good at actually predicting the outcome correctly and consistently.

## Conclusion

With more time to try and tune different algorithms, it might be possible to find a combination that improves our precision and recall, such that we can predict the fraudsters at Enron using their financial and email data. For now, our algorithms of choice just aren't that impressive.

This has been a really fun project, and I'm glad I had the opportunity to work on it. I hope that my answers satisfy your requirements. Thank you very much for taking the time and energy to read and review my work! I hope you have a wonderful morning/day/evening!

# Resources:


Udacity Forums:
https://discussions.udacity.com/t/how-to-start-the-final-project/177617/6

https://discussions.udacity.com/t/project-fear-strugging-with-machine-learning-project/198529/3

https://discussions.udacity.com/t/pickling-pandas-df/174753/2

https://discussions.udacity.com/t/how-to-go-about-p5/259206/10

https://discussions.udacity.com/t/do-we-just-remove-all-outliers/284364/4

https://discussions.udacity.com/t/verify-final-project-approach/244607

SKLearn Cheatsheet:
http://scikit-learn.org/stable/_static/ml_map.png

SKLearn Documentation (used a lot of this):
http://scikit-learn.org/stable/index.html

## Glossary if needed

__Salary__ - Reflects items such base salary, executive cash allowances, and benefits payments.

__Bonus__ - Reflects annual cash incentives paid based upon company performance. Also may include other retention payments.

__Long Term Incentive__ - Reflects long-term incentive cash payments from various long-term incentive programs designed to tie executive compensation to long-term success as measured against key performance drivers and business objectives over a multi-year period, generally 3 to 5 years.

__Deferred Income__ - Reflects voluntary executive deferrals of salary, annual cash incentives, and long-term cash incentives as well as cash fees deferred by non-employee directors under a deferred compensation arrangement. May also reflect deferrals under a stock option or phantom stock unit in lieu of cash arrangement.

__Deferral Payments__ - Reflects distributions from a deferred compensation arrangement due to termination of employment or due to in-service withdrawals as per plan provisions.

__Loan Advances__ - Reflects total amount of loan advances, excluding repayments, provided by the Debtor in return for a promise of repayment. In certain instances, the terms of the promissory notes allow for the option to repay with stock of the company.

__Other__ - Reflects items such as payments for severance, consulting services, relocation costs, tax advances and allowances for employees on international assignment (i.e. housing allowances, cost of living allowances, payments under Enron’s Tax Equalization Program, etc.). May also include payments provided with respect to employment agreements, as well as imputed income amounts for such things as use of corporate aircraft.

__Expenses__ - Reflects reimbursements of business expenses. May include fees paid for consulting services.

__Director Fees__ - Reflects cash payments and/or value of stock grants made in lieu of cash payments to non-employee directors.

__Exercised Stock Options__ - Reflects amounts from exercised stock options which equal the market value in excess of the exercise price on the date the options were exercised either through cashless (same-day sale), stock swap or cash exercises. The reflected gain may differ from that realized by the insider due to fluctuations in the market price and the timing of any subsequent sale of the securities.

__Restricted Stock__ - Reflects the gross fair market value of shares and accrued dividends (and/or phantom units and dividend equivalents) on the date of release due to lapse of vesting periods, regardless of whether deferred.

__Restricted Stock Deferred__ - Reflects value of restricted stock voluntarily deferred prior to release under a deferred compensation arrangement.